In [34]:
#импортруем необхожимые библиотеки
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandahouse
from scipy import stats

sns.set()

In [35]:
#подключение к бд
connection = {
'host': 'https://clickhouse.lab.karpov.courses',
'password': 'password',
'user': 'student',
'database': 'simulator_20220420'
}

In [36]:
#формируем запрос для групп 0 и 3
q = '''SELECT exp_group, 
    user_id,
    sum(action = 'like') as likes,
    sum(action = 'view') as views,
    likes/views as ctr
FROM {db}.feed_actions 
WHERE toDate(time) between '2022-04-04' and '2022-04-10'
    and exp_group in (0,3)
GROUP BY exp_group, user_id'''
df_0_3 = pandahouse.read_clickhouse(q, connection=connection)

In [37]:
df_0_3.head()

,exp_group,user_id,likes,views,ctr
0,3,115383,9,30,0.300000
1,3,123580,13,48,0.270833
2,0,4944,8,41,0.195122
3,0,4504,5,15,0.333333
4,0,121508,18,88,0.204545


In [38]:
#считаем общий ctr в контрольной группе
CTRcontrol_0 = df_0_3.query('exp_group == 0').likes.sum() / df_0_3.query('exp_group == 0').views.sum()

In [39]:
#посчитаем в обеих группах поюзерную метрику линеаризованных лайков
df_0_3['linearized_likes'] = df_0_3.apply(lambda x: x['likes'] - x['views']*CTRcontrol_0, axis=1)

In [40]:
# сравним t-тестом отличия в группах сначала по метрике ctr 
stats.ttest_ind(df_0_3.query('exp_group == 0')['ctr'],
                df_0_3.query('exp_group == 3')['ctr'],
                equal_var=False)

Ttest_indResult(statistic=-13.896870721904069, pvalue=1.055849414662529e-43)

In [41]:
#теперь сравним отличия по метрике linearized_likes
stats.ttest_ind(df_0_3.query('exp_group == 0')['linearized_likes'],
                df_0_3.query('exp_group == 3')['linearized_likes'],
                equal_var=False)

Ttest_indResult(statistic=-15.21499546090383, pvalue=5.4914249479687664e-52)

В обоих случаях различия фиксируются, однако во втором тесте p-value меньше.

In [45]:
#формируем запрос для групп 1 и 2
q = '''SELECT exp_group, 
    user_id,
    sum(action = 'like') as likes,
    sum(action = 'view') as views,
    likes/views as ctr
FROM {db}.feed_actions 
WHERE toDate(time) between '2022-04-04' and '2022-04-10'
    and exp_group in (1,2)
GROUP BY exp_group, user_id'''
df_1_2 = pandahouse.read_clickhouse(q, connection=connection)

In [46]:
df_1_2.head()

,exp_group,user_id,likes,views,ctr
0,1,109963,3,15,0.200000
1,1,26117,32,141,0.226950
2,1,138232,18,73,0.246575
3,1,26295,33,122,0.270492
4,1,18392,7,32,0.218750


In [47]:
#так же считаем общий ctr в контрольной группе
CTRcontrol_1 = df_1_2.query('exp_group == 1').likes.sum() / df_1_2.query('exp_group == 1').views.sum()

In [48]:
#так же посчитаем в обеих группах поюзерную метрику линеаризованных лайков
df_1_2['linearized_likes'] = df_1_2.apply(lambda x: x['likes'] - x['views']*CTRcontrol_1, axis=1)

In [49]:
stats.ttest_ind(df_1_2.query('exp_group == 1')['ctr'],
                df_1_2.query('exp_group == 2')['ctr'],
                equal_var=False)

Ttest_indResult(statistic=0.7094392041270485, pvalue=0.4780623130874935)

In [50]:
stats.ttest_ind(df_1_2.query('exp_group == 1')['linearized_likes'],
                df_1_2.query('exp_group == 2')['linearized_likes'],
                equal_var=False)

Ttest_indResult(statistic=6.122579994775972, pvalue=9.439432187037712e-10)

В данном случае t-тест зафиксировал разницу только после линеаризации лайков.